<a href="https://colab.research.google.com/github/ArifAygun/Magnimind-ML/blob/main/AA_5_02_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import metrics
%matplotlib inline

# Bank Marketing Data - A Decision Tree Approach

## Aim:
The aim of this attempt is to predict if the client will subscribe (yes/no) to a term deposit, by building a classification model using Decision Tree.
### Step 1: Load the data
- Load `bank.csv' data
- Check the first five observations
- Check if there are any null values

In [23]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/My Drive/Datasets/

bank=pd.read_csv('bank.csv')
bank.head()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Datasets


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


## Summay of data

### Categorical Variables :
**[1] job      :** admin,technician, services, management, retired, blue-collar, unemployed, entrepreneur,
               housemaid, unknown, self-employed, student
<br>**[2] marital  :** married, single, divorced
<br>**[3] education:** secondary, tertiary, primary, unknown
<br>**[4] default  :** yes, no
<br>**[5] housing  :** yes, no
<br>**[6] loan     :** yes, no 
<br>**[7] deposit  :** yes, no ** (Dependent Variable)**
<br>**[8] contact  :** unknown, cellular, telephone
<br>**[9] month    :** jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec
<br>**[10] poutcome:** unknown, other, failure, success

### Numerical Variables:
**[1] age 
<br>[2] balance
<br>[3] day
<br>[4] duration
<br>[5] campaign
<br>[6] pdays
<br>[7] previous **

In [24]:
# Check if the data set contains any null values - Nothing found!
df_datatype = pd.concat([bank.dtypes, bank.nunique(),
                         round(bank.nunique()*100/len(bank), 2),
                         bank.isnull().sum(),
                         round(bank.isnull().sum()*100/len(bank),2)],
                        axis=1)

df_datatype.columns = ['data type',
                       'No. of categories',
                       'No. of categories %',
                       'missing values',
                       'missing values %']

df_datatype.sort_values(by='missing values %',
                        ascending=False, inplace=True)
df_datatype

,data type,No. of categories,No. of categories %,missing values,missing values %
age,int64,76,0.68,0,0.0
day,int64,31,0.28,0,0.0
poutcome,object,4,0.04,0,0.0
previous,int64,34,0.30,0,0.0
pdays,int64,472,4.23,0,0.0
campaign,int64,36,0.32,0,0.0
duration,int64,1428,12.79,0,0.0
month,object,12,0.11,0,0.0
contact,object,3,0.03,0,0.0
job,object,12,0.11,0,0.0


In [25]:
bank.shape

(11162, 17)

In [26]:
bank.describe()

,age,balance,day,duration,campaign,pdays,previous
count,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000
mean,41.231948,1528.538524,15.658036,371.993818,2.508421,51.330407,0.832557
std,11.913369,3225.413326,8.420740,347.128386,2.722077,108.758282,2.292007
min,18.000000,-6847.000000,1.000000,2.000000,1.000000,-1.000000,0.000000
25%,32.000000,122.000000,8.000000,138.000000,1.000000,-1.000000,0.000000
50%,39.000000,550.000000,15.000000,255.000000,2.000000,-1.000000,0.000000
75%,49.000000,1708.000000,22.000000,496.000000,3.000000,20.750000,1.000000
max,95.000000,81204.000000,31.000000,3881.000000,63.000000,854.000000,58.000000


### Step 2: Transformer
- Create a trasnformer pipeline for numeric and categorical features. numerical features will be imputed and scaled. Categorical features will be imputed and encoded
- Create a Column transformer

In [27]:
# Make a copy for parsing
b_data = bank.copy()

numeric_features = ['age','balance','day','duration','pdays','previous']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['job', 'marital','education','default','housing','loan','contact','month','poutcome']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['age', 'balance', 'day', 'duration', 'pdays',
                                  'previous']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['job', 'marital', 'education', 'default',
                                  'housing', 'loan', 'contact', 'month',
                                  'poutcome'])])

### Step 3: Classifier
- Create a pipeline for the decision tree classifier as well as the transformer
- Encode the target variable using `LabelEncoder`

In [28]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier',   DecisionTreeClassifier(random_state=1, max_depth=2))])

label_encoder=LabelEncoder()
b_data['deposit_cat']=label_encoder.fit_transform(b_data['deposit']) 
b_data.drop('deposit', axis=1, inplace=True)

### Step 4: Model
- Create a pipeline for the decision tree classifier as well as the transformer
- Encode the target variable using `LabelEncoder`

In [29]:
# Train-Test split: 20% test data
X = b_data.drop('deposit_cat', 1)
y = b_data.deposit_cat
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 50)
clf.fit(X_train,y_train)
score_train = clf.score(X_train, y_train)
print("Training score: ",score_train)
score_test = clf.score(X_test, y_test)
print("Testing score: ",score_test)

Training score:  0.7285250307985217
Testing score:  0.7268248992386923


<ipython-input-29-75fa80a57153>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = b_data.drop('deposit_cat', 1)


In [30]:
# A slightly different solution
b_data = bank.copy()
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

label_encoder=LabelEncoder()
b_data['deposit_cat']=label_encoder.fit_transform(b_data['deposit']) 
b_data.drop('deposit', axis=1, inplace=True)


X = b_data.drop('deposit_cat', 1)
y = b_data.deposit_cat

categorical = X.dtypes == object
print(categorical)
print(~categorical)


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])


categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, ~categorical),
        ('cat', categorical_transformer, categorical)])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier',   DecisionTreeClassifier(random_state=1, max_depth=2))])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 50)
clf.fit(X_train,y_train)
score_train = clf.score(X_train, y_train)
print("Training score: ",score_train)
score_test = clf.score(X_test, y_test)
print("Testing score: ",score_test)

<ipython-input-30-920a2c2dd82e>:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = b_data.drop('deposit_cat', 1)


age          False
job           True
marital       True
education     True
default       True
balance      False
housing       True
loan          True
contact       True
day          False
month         True
duration     False
campaign     False
pdays        False
previous     False
poutcome      True
dtype: bool
age           True
job          False
marital      False
education    False
default      False
balance       True
housing      False
loan         False
contact      False
day           True
month        False
duration      True
campaign      True
pdays         True
previous      True
poutcome     False
dtype: bool
Training score:  0.7285250307985217
Testing score:  0.7268248992386923


In [34]:
# Import necessary libraries
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Load data and create label encoder
bank = pd.read_csv('bank.csv')
label_encoder = LabelEncoder()

# Encode the target variable and drop the original column
bank['deposit_cat'] = label_encoder.fit_transform(bank['deposit'])
bank.drop('deposit', axis=1, inplace=True)

# Split the data into input features and target variable
X = bank.drop('deposit_cat', axis=1)
y = bank['deposit_cat']

# Identify categorical and numerical features
categorical = X.dtypes == object
numerical = ~categorical

# Define preprocessing pipelines for categorical and numerical features
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Combine the pipelines into a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('cat', categorical_pipeline, categorical),
    ('num', numerical_pipeline, numerical)])

# Create the final pipeline by adding the preprocessor and the classifier
clf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=1, max_depth=2))])

# Split the data into training and testing sets and fit the classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)
clf.fit(X_train, y_train)

# Evaluate the model on training and testing sets
score_train = clf.score(X_train, y_train)
score_test = clf.score(X_test, y_test)
print("Training score: ", score_train)
print("Testing score: ", score_test)


Training score:  0.7285250307985217
Testing score:  0.7268248992386923
